In [84]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import warnings

warnings.filterwarnings("ignore") 

In [85]:
file_path = '/home/daniar/Vs_Code/Kuliah/Semester 4/TWS/Train Model/data/Jawa Timur_Kota Surabaya.csv'
df = pd.read_csv(file_path)

In [86]:
df['TANGGAL'] = pd.to_datetime(df['TANGGAL'])
df = df.set_index('TANGGAL')

In [87]:
features_to_predict_original = ['TN', 'TX', 'TAVG', 'RH_AVG', 'RR', 'SS']
all_relevant_columns = features_to_predict_original + ['DDD_CAR']

In [88]:
for col in features_to_predict_original:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [89]:
compass_to_degrees = {
    "N": 0, "NNE": 22.5, "NE": 45, "ENE": 67.5,
    "E": 90, "ESE": 112.5, "SE": 135, "SSE": 157.5,
    "S": 180, "SSW": 202.5, "SW": 225, "WSW": 247.5,
    "W": 270, "WNW": 292.5, "NW": 315, "NNW": 337.5,
    "C": 0
}

def direction_to_sin_cos(direction):
    if pd.isna(direction) or direction not in compass_to_degrees:
        return np.nan, np.nan
    degree = compass_to_degrees[direction]
    rad = np.deg2rad(degree)
    return np.sin(rad), np.cos(rad)

if 'DDD_CAR' in df.columns:
    df['DDD_CAR'] = df['DDD_CAR'].astype(str).str.strip().str.upper()
    df['WindDir_sin'], df['WindDir_cos'] = zip(*df['DDD_CAR'].map(direction_to_sin_cos))
    df = df.drop(columns=['DDD_CAR'])
else:
    print("\nDDD_CAR column not found. Skipping wind direction encoding.")

In [90]:
features_for_modeling = features_to_predict_original[:] # Create a copy
if 'WindDir_sin' in df.columns: 
    features_for_modeling.extend(['WindDir_sin', 'WindDir_cos'])

for col in features_for_modeling:
    if col in df.columns:
        # Using linear interpolation for numeric columns
        df[col] = df[col].interpolate(method='linear')
        # Fill any remaining NaNs at the beginning or end
        df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
    else:
        print(f"Warning: Column {col} not found for NaN handling after encoding.")

In [91]:
df.dropna(subset=features_for_modeling, inplace=True)

In [92]:
def find_best_arima_order(series, p_range, d_range, q_range):
    best_aic = np.inf
    best_order = None
    
    if series.ndim > 1:
        series = series.iloc[:,0] # Or series.squeeze()

    for d in d_range:
        for p in p_range:
            for q in q_range:
                try:
                    model = ARIMA(series, order=(p,d,q))
                    results = model.fit()
                    if results.aic < best_aic:
                        best_aic = results.aic
                        best_order = (p,d,q)
                except Exception as e:
                    continue
    return best_order

In [93]:
p_values = range(0, 4) 
d_values = range(0, 2) 
q_values = range(0, 4) 

forecast_horizon = 21
rmse_scores = {}
future_forecasts_dict = {}

train_df = df.iloc[:-forecast_horizon]
test_df = df.iloc[-forecast_horizon:]

In [94]:
import joblib
import os

# Direktori penyimpanan model
model_save_dir = 'saved_arima_models'
os.makedirs(model_save_dir, exist_ok=True)

for feature in features_for_modeling:
    if feature not in df.columns or df[feature].isnull().all():
        print(f"\nSkipping feature '{feature}' due to missing data or column not found.")
        continue
    
    series_train = train_df[feature].astype(np.float64)
    series_test = test_df[feature].astype(np.float64)

    if len(series_train) < (max(p_values) + max(d_values) + max(q_values) + 5) or len(series_train) < 10:
        print(f"Not enough data points for feature {feature} to reliably fit ARIMA. Skipping.")
        rmse_scores[feature] = np.nan
        future_forecasts_dict[feature] = [np.nan] * forecast_horizon
        continue

    # Cari best ARIMA order
    best_order = find_best_arima_order(series_train, p_values, d_values, q_values)
    if best_order is None:
        print(f"Could not find a suitable ARIMA order for {feature}. Using default (1,1,1).")
        best_order = (1, 1, 1)

    # Train on train set dan evaluasi dengan RMSE
    try:
        model_eval = ARIMA(series_train, order=best_order)
        model_eval_fit = model_eval.fit()
        predictions_test = model_eval_fit.forecast(steps=forecast_horizon)

        if len(predictions_test) == len(series_test):
            rmse = np.sqrt(mean_squared_error(series_test, predictions_test))
            rmse_scores[feature] = rmse
        else:
            print(f"Warning: Length mismatch for {feature}. Test: {len(series_test)}, Pred: {len(predictions_test)}. Skipping RMSE.")
            rmse_scores[feature] = np.nan

    except Exception as e:
        print(f"Error during evaluation model fitting or prediction for {feature}: {e}")
        rmse_scores[feature] = np.nan

    # Retrain dengan full data dan forecast masa depan
    full_series = df[feature].astype(np.float64)
    try:
        model_future = ARIMA(full_series, order=best_order)
        model_future_fit = model_future.fit()
        future_forecast_values = model_future_fit.forecast(steps=forecast_horizon)
        future_forecasts_dict[feature] = future_forecast_values

    except Exception as e:
        print(f"Error during final model fitting or future forecasting for {feature}: {e}")
        future_forecasts_dict[feature] = [np.nan] * forecast_horizon


In [95]:
# print("\n--- RMSE Scores for Each Feature (on Test Set) ---")
# if rmse_scores:
#     for feature, rmse in rmse_scores.items():
#         if pd.notna(rmse):
#             print(f"RMSE for {feature}: {rmse:.4f}")
#         else:
#             print(f"RMSE for {feature}: Not available")
# else:
#     print("No RMSE scores were calculated.")


In [97]:
# Create a DataFrame for future forecasts
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=forecast_horizon, freq='D')
forecast_df = pd.DataFrame(index=future_dates)

print("\n--- Forecasts for the Next 21 Days ---")
if future_forecasts_dict:
    for feature, forecasts in future_forecasts_dict.items():
        forecast_df[feature] = round(forecasts,2)
    print(forecast_df)
else:
    print("No forecasts were generated.")


--- Forecasts for the Next 21 Days ---
               TN     TX   TAVG  RH_AVG    RR    SS  WindDir_sin  WindDir_cos
2025-05-30  26.40  33.35  29.22   79.34  4.20  3.53         0.19         0.59
2025-05-31  26.17  33.37  29.00   80.05  3.92  3.68         0.23         0.57
2025-06-01  26.08  33.36  28.82   80.89  1.64  3.79         0.25         0.68
2025-06-02  26.04  33.20  28.83   80.93  5.32  3.86         0.27         0.58
2025-06-03  26.02  33.14  28.76   81.31  2.04  3.90         0.28         0.54
2025-06-04  26.00  33.10  28.80   81.20  3.36  3.94         0.28         0.66
2025-06-05  25.99  33.06  28.75   81.41  4.59  3.96         0.29         0.61
2025-06-06  25.98  33.04  28.79   81.29  1.46  3.97         0.29         0.53
2025-06-07  25.97  33.02  28.76   81.43  4.94  3.98         0.29         0.64
2025-06-08  25.95  33.01  28.79   81.33  2.79  3.99         0.29         0.64
2025-06-09  25.94  33.00  28.76   81.42  2.61  3.99         0.29         0.53
2025-06-10  25.93  32.99